In [5]:
import copy
import itertools

# TEMA 3

## Utilidades

In [6]:
# Devuelve el exponente lider del polinomio f
def exp(f):
    return f.exponents()[0]

In [7]:
# Devuelve el maximo en cada posición de alpha y beta
def lcm(alpha, beta):
    return alpha.emax(beta)

In [8]:
# Calcula el S-polinomio de f y g en R
def S(R, f, g):
    alpha = exp(f)
    beta = exp(g)
    gamma = lcm(alpha, beta)
    delta1 = gamma.esub(alpha)
    delta2 = gamma.esub(beta)
    return g.lc() * R.monomial(*delta1) * f - f.lc() * R.monomial(*delta2) * g

## Algoritmo de división


In [9]:
# Divide f entre F en R, devuelve los coef y el resto
def division(R, f, F):
    # Polinomio actual
    p = copy.deepcopy(f)
    # Resto
    r = 0
    # Nº de polinomios
    s = len(F)
    # Coeficientes (q_i)
    Q = []
    for i in range(s):
        Q.append(0)
    # Hasta que p sea 0
    while p != 0:
        i = 0
        step = 0
        # Si se puede dividir por algun f_i
        while i < s and step == 0:
            # Calculamos gamma (exp(p) = exp(f_i) + gamma)
            gamma = exp(p).esub(exp(F[i]))
            # Gamma debe tener todas las posiciones no negativas
            if all(map(lambda x : x >= 0, gamma)):
                # Coef
                coef = p.lc() / F[i].lc()
                # Actualizamos q_i y p
                Q[i] += coef * R.monomial(*gamma)
                p -= coef * R.monomial(*gamma) * F[i]
                step = 1
            else:
                i += 1
        # Si no es divisible entre ningun f_i añadimos al resto
        if step == 0:
            r += p.lt()
            p -= p.lt()
    return r, Q

Ejemplo:

In [10]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
f = x^2*y+x*y^2+y^2
f1 = x*y-1
f2 = y^2-1
print(division(R, f, [f2, f1]))
print(division(R, f, [f1, f2]))

(2*x + 1, [x + 1, x])
(x + y + 1, [x + y, 1])


## Criterio de Buchberger (comprobar base)

In [11]:
# Comprueba si el conjunto de generadores G es base de Groebner en R
def criterio_buchberger(R, G):
    t = len(G)
    for i in range(t):
        for j in range(i + 1, t):
            # G base de Groebner sii rem(S(g_i, g_j), G) = 0
            r, Q = division(R, S(R, G[i], G[j]), G)
            if r != 0:
                return false
    return true

In [12]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
I = R.ideal([x*y^2 + x*y, y^5, y*z^4, x^2*y*z])
print(criterio_buchberger(R, I.groebner_basis()))
print(criterio_buchberger(R, [x*y^2 + x*y, y^5, y*z^4, x^2*y*z]))

True
False


## Algoritmo de Buchberger (obtener base de Groebner)

In [13]:
# Calcula una basa de Groebner para el ideal generado por F en R
# NO recomendable para F grandes
def algoritmo_buchberger(R, F):
    G = copy.deepcopy(F)
    while true:
        G_prime = copy.deepcopy(G)
        for i in range(len(G_prime)):
            for j in range(len(G_prime)):
                if i != j:
                    r, _ = division(R, S(R, G_prime[i], G_prime[j]), G_prime)
                    if r != 0:
                        G = G + [r]
        if G_prime == G:
            break
    return G    

In [14]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
F = [x^2*y - 1, x*y^2 - x]
B = algoritmo_buchberger(R,F)
print(criterio_buchberger(R, B))

True


## Base de Groebner reducida

In [15]:
# Reduce la base de Groebner G en R
def groebner_reducida(R, B):
    G = copy.deepcopy(B)
    i = 0
    for _ in range(len(G)):
        # rem(g, [G \ {g}])
        r, Q = division(R, G[i], G[:i] + G[i+1:])
        # Si r != 0, lo añadimos
        if r != 0:
            # lc(r) = 1
            r /= r.lc()
            G =  G[:i] + [r] + G[i+1:]
            i += 1
        # Si r == 0, lo descartamos
        else:
            G = G[:i] + G[i+1:]
    return G

In [16]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
F = [x^2*y - 1, x*y^2 - x]
B = algoritmo_buchberger(R,F)
print(groebner_reducida(R,B))
print(R.ideal(F).groebner_basis())

[x^2 - y, y^2 - 1]
[x^2 - y, y^2 - 1]


In [17]:
R.<x,y,z,w> = PolynomialRing(QQ, ["x", "y", "z", "w"], order = "lex")
F = [3*x-6*y-2*z, 2*x-4*y+4*w, x-2*y-z-w]
B = algoritmo_buchberger(R,F)
print(groebner_reducida(R, B))
print(R.ideal(F).groebner_basis())

[x - 2*y + 2*w, z + 3*w]
[x - 2*y + 2*w, z + 3*w]


## Base de Groebner reducida

In [18]:
# Devuelve la base de groebner reducida de un ideal generado por F
def groebner_base(R, F):
    # Obtenemos base de Groebner
    B = algoritmo_buchberger(R, F)
    # La reducimos
    return groebner_reducida(R, B)

In [19]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
f1 = x*y-1
f2 = y^2-1
B = groebner_base(R, [f1, f2])
print(B)

[y^2 - 1, x - y]


## Comprobar si un polinomio pertenece a un ideal

In [20]:
# Si un polinomio f pertenece a un ideal generado por F en R
def check_in_ideal(R, f, F):
    # Otras alternativas
    # -------------------
    # I = R * F
    # f in I
    # I.reduce(f) == 0
    # B = I.groebner_basis()
    # -------------------
    # Base de groebner
    if 1 in F: return True
    B = R.ideal(F).groebner_basis()
    # Pertenece si el resto es 0
    r, _ = division(R, f, B)
    return r == 0

In [21]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
f = x^2*y^2 + x^2*y - y + 1
F = [x*y^2 + x, x*y-y^3]
print(check_in_ideal(R, f, F))
print(check_in_ideal(R, (F*R).groebner_basis()[0], F))

False
True


## Ej 3.2


In [22]:
def ej3_2(ring, order, reverse):
    R.<x,y,z> = PolynomialRing(ring, ["x", "y", "z"], order="deglex")
    f = x^7*y^2 + x^3*y^2-y+1
    f1 = x*y^2-x
    f2 = x-y^3
    F = [f1, f2]
    if reverse:
        F = [f2, f1]
    print(division(R, f, F))

In [23]:
ej3_2(QQ, "deglex", false)
ej3_2(QQ, "lex", false)
ej3_2(QQ, "deglex", true)
ej3_2(QQ, "lex", true)
ej3_2(GF(2), "deglex", false)
ej3_2(GF(3), "deglex", false)
ej3_2(GF(4), "deglex", false)
ej3_2(GF(5), "deglex", false)

(x^7 + x^3 - y + 1, [x^6 + x^2, 0])
(x^7 + x^3 - y + 1, [x^6 + x^2, 0])
(x^7 + x^3 - y + 1, [0, x^6 + x^2])
(x^7 + x^3 - y + 1, [0, x^6 + x^2])
(x^7 + x^3 + y + 1, [x^6 + x^2, 0])
(x^7 + x^3 - y + 1, [x^6 + x^2, 0])
(x^7 + x^3 + y + 1, [x^6 + x^2, 0])
(x^7 + x^3 - y + 1, [x^6 + x^2, 0])


## Ej 3.4

In [24]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "degrevlex")
f = x^3-x^2*y-x^2*z
f1 = x^2*y-z
f2 = x*y-1
r1, Q1 = division(R, f, [f1,f2])
r2, Q2 = division(R, f, [f2,f1])
print("Resto: ", r1, ", coeficientes: ", Q1)
print("Resto: ", r2, ", coeficientes: ", Q2)
r = r1 - r2
print(check_in_ideal(R, r, [f1, f2]))
print(groebner_base(R, [f1,f2]))
print(division(R, r, [f1,f2]))

('Resto: ', x^3 - x^2*z - z, ', coeficientes: ', [-1, 0])
('Resto: ', x^3 - x^2*z - x, ', coeficientes: ', [-x, 0])
True
[x - z, y*z - 1]
(x - z, [0, 0])


# Tema 4

## Intersección con anillo

In [25]:
# Si R = F[x_(l+1), ..., x_n] calcula F interseccion R
# No hacer si F es generador de ideal I, hacer con base Groebner
def interseccion_reduccion(R, l, F):
    assert l >= 0
    interseccion = []
    # Para cada polinomio de F
    for f in F:
        insertar = True
        # Miramos en cada monomio de f
        for exp in f.exponents():
            # Miramos las posiciones de los exponentes > 0
            for pos in exp.nonzero_positions():
                # Si alguno está en F[x_1, ..., x_l] no lo añadimos
                if pos <= (l-1):
                    insertar = False
                    break
            # Cortamos antes
            if not insertar:
                break
        # Insertamos
        if insertar:
            interseccion.append(copy.deepcopy(R(f)))
    return interseccion

In [26]:
# I interseccion R = F[x_(l+1), ..., x_n] siendo I un ideal
def interseccion_reduccion_ideal(R, l, I):
    assert l >= 0
    F = I.gens()
    if 1 in F: return [R(1)]
    B = ideal(F).groebner_basis()
    return interseccion_reduccion(R, l, B)

In [27]:
R.<x,y> = PolynomialRing(QQ, 2, "xy", order = "lex")
F = [x^2, y^3, y^4 + y, x*y + x^2, y]
print(interseccion_reduccion(R, 1, ideal(F).groebner_basis()))
print(interseccion_reduccion_ideal(R, 1, ideal(F)))

[y]
[y]


In [28]:
R.<x,y> = PolynomialRing(GF(3), ["x", "y"], order = "lex")
I = ideal(-x^2*y-y^3-x^2+x*y+y, x^2*y-y^3-x*y-y^2+y)
print(interseccion_reduccion_ideal(R, 1, I))

[y^7 - y^6 + y^3 + y]


## Interseccion de ideales

In [29]:
# Base de groebner de la interseccion de I = <F> con J = <G> 
# en R = F[x_1, ..., x_n]
def ideal_interseccion(R, F, G, order):
    # A = F[t, x_1, .., x_n]
    _, base = R.construction()
    A = PolynomialRing(base, ["aux"] + list(R.gens()), order = order)
    t = A.gen(0)
    # ideal(H) = <t*f_1, ..., t*f_s, (1-t)*g_1, ..., (1-t)*g_t>
    H = []
    for f in F:
        H = H + [t * f]
    for g in G:
        H = H + [(1-t) * g]
    # K
    if 1 in H: return [1]
    K = A.ideal(H)
    # Interseccion G_H F[x_1, ..., x_n]
    return interseccion_reduccion_ideal(R, 1, K)

In [30]:
R.<t,x,y> = PolynomialRing(GF(3), ["t","x", "y"], order = "lex")
F = [-x^3-x*y^2, -x*y^2-y^3+x^2]
G = [y^2-x+y+1, x^2+x*y+y^2+x, x*y-y^2-y]
print(ideal_interseccion(R, F, G, "lex"))

[x^2 - y^5 - y^3, x*y^2 - y^5, y^7 + y^6 - y^5]


## Implicitación polinomial (infinito)

In [31]:
# R = F[t_1, ..., t_r] cuerpo infinito, phi(a) = (F(a)), F = (f1,...,f_n)
# devuelve J con V(J) la menor variedad que contiene a phi(F^r)
def implicitacion_polinomial(R, F, order):
    _, base = R.construction()
    n_vars = ["x" + str(i+1) for i in range(len(F))]
    # A = F[t_1, ..., t_r, x_1, ..., x_n]
    A = PolynomialRing(base, list(R.gens()) + n_vars, order = order)
    # I = <x_1 - f_1, ..., x_n - f_n>
    I = A.ideal([x_i - f_i for (x_i, f_i) 
               in zip(A.gens()[len(R.gens()):], F)])
    return interseccion_reduccion_ideal(A, len(R.gens()), I)

In [32]:
R.<u,v> = PolynomialRing(QQ, ["u", "v"], order = "lex")
F = [u^2-v^2, u^2+v^2+v, -u*v+u+v]
print(implicitacion_polinomial(R, F, "lex"))

[x1^4 + 3*x1^3 - 2*x1^2*x2^2 + 8*x1^2*x2 + 8*x1^2*x3^2 - 28*x1^2*x3 + 14*x1^2 - x1*x2^2 - 16*x1*x2*x3 + 22*x1*x2 + 12*x1*x3^2 - 26*x1*x3 + 5*x1 + x2^4 - 10*x2^3 - 8*x2^2*x3^2 + 44*x2^2*x3 - 64*x2*x3^2 + 10*x2*x3 + 16*x3^4 + 16*x3^3 - 5*x3^2]


## Implicitación racional (infinito)

In [33]:
# R = F[t_1, ..., t_r] cuerpo infinito, phi(a) = (F(a)), 
# F = (f1/q_1,...,f_n/q_n) devuelve J con V(J) 
# la menor variedad que contiene a phi(F^r)
def implicitacion_racional(R, F, Q, order):
    _, base = R.construction()
    n_vars = ["x" + str(i+1) for i in range(len(F))]
    # A = F_q[y, t_1, ..., t_r, x_1, ..., x_n]
    A = PolynomialRing(base, ["aux"] + list(R.gens()) + n_vars, 
                        order = order)
    # I = <q1 * x_1 - f_1, ..., q_n * x_n - f_n, 1 - q_1 * ... q_n * y>
    G = [q_i * x_i - f_i for (q_i, x_i, f_i) in 
         zip(Q, A.gens()[1+len(R.gens()):], F)]
    f_aux = A.gens()[0]
    for q_i in Q:
        f_aux *= q_i
    G = G + [1 - f_aux]
    # Interseccion
    return interseccion_reduccion_ideal(A, len(R.gens()) + 1, A.ideal(G))

In [34]:
R.<t> = PolynomialRing(QQ, ["t"], order = "lex")
F = [1 - t^2, 2*t]
Q = [1+t^2, 1+t^2]
print(implicitacion_racional(R, F, Q, "lex"))

[x1^2 + x2^2 - 1]


In [35]:
# Parcial 2 - 2
R.<t> = PolynomialRing(QQ, ["t"], order = "lex")
F = [2*t^2, 2*t^3]
Q = [1 + t^2, 1 + t^2]
print(implicitacion_racional(R, F, Q, "lex"))

[x1^3 + x1*x2^2 - 2*x2^2]


## Implicitacion polinomial (finito)

In [36]:
# R = F_q[t_1, ..., t_r] cuerpo finito, phi(a) = (F(a)), F = (f1,...,f_n)
# devuelve J con V(J) la menor variedad que contiene a phi(F^r_q)
def implicitacion_polinomial_finito(R, F, order):
    _, base = R.construction()
    q = base.cardinality()
    n_vars = ["x" + str(i+1) for i in range(len(F))]
    # A = F[t_1, ..., t_r, x_1, ..., x_n]
    A = PolynomialRing(base, list(R.gens()) + n_vars, order = order)
    # I = <x_1 - f_1, ..., x_n - f_n>
    G = [x_i - f_i for (x_i, f_i) in zip(A.gens()[len(R.gens()):], F)]
    G += [x_i^q]
    return interseccion_reduccion_ideal(A, len(R.gens()), A.ideal(G))

In [37]:
R.<t> = PolynomialRing(GF(3), ["t"], order = "lex")
F = [t^2, t+1]
print(implicitacion_polinomial_finito(R, F, "lex"))

[x1 - x2^2 - x2 - 1, x2^3]


# Tema 5

## Polinomio en radical

In [38]:
# Si el polinomio f pertenece al radical generado por F en R
def polinomio_en_radical(R, f, F, order):
    _, base = R.construction()
    # A = F[x_1, ..., x_n, y]
    A = PolynomialRing(base, list(R.gens()) + ["aux"], order = order)
    # J = <I> + <1 - f*y>
    J = A.ideal(F + [1 - f * A.gen(len(R.gens()))])
    # Si 1 en B => J = A
    if 1 in J.gens(): return True
    B = J.groebner_basis()
    return 1 in B

In [39]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
F = [x^3, y^3, x*y*(x+y)]
f = x+y
print(polinomio_en_radical(R, f, F, "lex"))
i = 0
while true:
    if check_in_ideal(R, f^i, F):
        print(i, f^i)
        break
    i += 1
J = [x+z, x^2*y, x-z^2]
g = x^2+3*x*z
print(polinomio_en_radical(R, g, J, "lex"))

True
(3, x^3 + 3*x^2*y + 3*x*y^2 + y^3)
False


In [40]:
# Parcial 2 - Ejercicio 4
R.<x,y,z> = PolynomialRing(QQ, 3, "xyz", order = "lex")
f1 = x^2-y*z
f2 = y^2-x*y*z
F = [f1^2, f1*f2, f2^2]
G = [f1, f2]
f = y*z^2-x*y
print(polinomio_en_radical(R, f, F, "lex"))
print(check_in_ideal(R, f, G))

True
False


## Cociente

In [41]:
# Calcula base de Groebner del cociente I : f con I = <F> en R
def cociente_ideal_polinomio(R, F, f, order):
    # I interseccion <g> = <h_1, ..., h_s>
    H = ideal_interseccion(R, F, [f], order)
    K = [h_i / f for h_i in H]
    if 1 in K: return [1]
    return R.ideal(K).groebner_basis()

In [42]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
F = [(x+y)^2*(x-y)*(x+z^2)]
g = (x+z^2)^3*(x-y)*(z+y)
print(cociente_ideal_polinomio(R, F, g, "lex"))

[x^2 + 2*x*y + y^2]


In [43]:
# Calcula generadores del cociente I : J con I = <F> en R
def cociente_ideales(R, F, G, order):
    cocientes = [cociente_ideal_polinomio(R, F, g, order) for g in G]
    res = copy.deepcopy(cocientes[0])
    for cociente in cocientes[1:]:
        res = ideal_interseccion(R, res, cociente, order)
    return res

In [44]:
# Parcial 2 - Ejercicio 3
R.<x,y> = PolynomialRing(GF(5), ["x", "y"], order = "lex")
F = [x^2+x*y]
G = [x+y, y^2-x^2]
print(cociente_ideales(R, F, G, "lex"))
G_b = R.ideal(G).groebner_basis()
print(cociente_ideales(R, F, G_b, "lex"))

[x]
[x]


## Saturación

In [45]:
def saturacion_ideal_polinomio(R, F, f, order):
    _, base = R.construction()
    # A = F[y, x_1, ..., x_n]
    A = PolynomialRing(base, ["aux"] + list(R.gens()), order = order)
    aux = A.gen(len(R.gens()))
    # I_tilde = I + <1 - y * g>
    F_tilde = F + [1-aux*f]
    if 1 in F_tilde: return [1]
    return interseccion_reduccion_ideal(A, 1, A.ideal(F_tilde))

In [46]:
# Calcula generadores de la saturacion I : J^inf con I = <F> en R
def saturacion_ideales(R, F, G, order):
    saturaciones = [saturacion_ideal_polinomio(R, F, g, order) for g in G]
    res = copy.deepcopy(saturaciones[0])
    for saturacion in saturaciones[1:]:
        res = ideal_interseccion(R, res, saturacion, order)
    return res

In [47]:
R.<x,y> = PolynomialRing(GF(5), ["x", "y"], order = "lex")
F = [x^2+x*y]
G = [x+y, y^2-x^2]
print(saturacion_ideales(R, F, G, "lex"))

[x, y^4 + y^3 - y - 1]


# Tema 6

## Utilidades

In [48]:
# Conjunto exp de una lista de polinomios F
def exp(F):
    s = []
    for f in F:
        s = s + [f.exponents()[0]]
    return list(set(s))

In [49]:
R.<x,y> = PolynomialRing(QQ, 2, "xy", order = "lex")
F = [x*y^2 + 3*y, x^4 + y^3 - x^2*y^2 + 3*y + x*y^2]
print(exp(F))

[(1, 2), (4, 0)]


In [50]:
# Calcula top_m(alpha)
def top(m, alpha):
    return [i+1 for i in range(0, len(alpha)) if alpha[i] >= m]

In [51]:
print(top(3, (4,2,1)))

[1]


In [52]:
# Calcula sh_m(alpha)
def sh(m, alpha):
    ind = top(m, alpha)
    return [m if i+1 in ind else alpha[i] for i in range(0, len(alpha))]

In [53]:
print(sh(3, (5, 4, 1)))

[3, 3, 1]


In [54]:
# Calcula R_m
def R_m(m, F, n):
    if any(sh(m, alpha) not in F for alpha in F):
        print("Error: no se cumple la condición")
        return None
    return [f for f in F if all(f[i] <= m for i in range(0, n))]

In [55]:
print(R_m(2, [[1, 2, 3], [2, 2, 1], [1,2,2]], 3))

[[2, 2, 1], [1, 2, 2]]


## Función T

In [56]:
# Si un sigma pertenece a T(E) con E = X + N^n
def check_supp(X, sigma):
    return all(any(alpha[pos] != 0 for pos in sigma) for alpha in X)

In [57]:
# Calcula T(E) con E ideal de N^n
def T(E, n):
    T_E = []
    # Para cada combinación desde tamaño 1 hasta tamaño n
    for k in range(0, n):
        for sigma in itertools.combinations(range(0, n), k + 1):
            # Si existe un sigma
            if not E or check_supp(E, sigma):
                T_E = T_E + [[x + 1 for x in sigma]]
    return T_E

In [58]:
E = [(0, 1, 2), (2, 2, 1), (3, 3, 1), (0, 2, 1)]
print(T(E, 3))
print(T([], 3))
print(T([(0, 0, 0), (1, 2, 3)], 3))

[[2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
[[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
[]


In [59]:
# Calcula T(exp(I)) con exp(I) = exp(G) + N^n y I = <F>
def T_I(R, F):
    if 1 in F: return []
    G = R.ideal(F).groebner_basis()
    if 1 in G: return []
    return T(exp(G), len(R.gens()))

In [60]:
R.<x,y> = PolynomialRing(QQ, 2, "xy", order = "lex")
F = [x*y^2 + x, x^3*y]
print(T_I(R, F))

[[1], [1, 2]]


## Dimensión de un ideal

In [69]:
# Si un ideal I = <F> calcula dim(exp(I)) en R
def dim_ideal(R, F):
    n = len(R.gens())
    # Si I = R
    if 1 in F: return 0
    # Calculamos T(exp(B)) = T(exp(I))
    B = R.ideal(F).groebner_basis()
    # Si I = R o I = []
    if 1 in B: return 0
    if 0 in B: return n
    # Conjunto de exponentes
    X = exp(B)
    # Para cada combinación desde tamaño 1 hasta tamaño n
    for k in range(0, n):
        for sigma in itertools.combinations(range(0, n), k + 1):
            # Si existe un sigma
            if check_supp(X, sigma):
                return n - len(sigma)

In [73]:
R.<x,y> = PolynomialRing(QQ, 2, "xy", order = "deglex")
F = [y*x^2, y^2*x^2 + x*y + 1]
print(dim_ideal(R, F))

0


## Grado del polinomio de Hilbert

In [63]:
# Devuelve el grado del polinomio de Hilbert de un ideal I (sobre R/I)
# R tiene que tener orden graduado!
def grado_Hilbert(R, F):
    # HF_{R/I}(s) = HF_{exp(I)}(s) = h(s) (s >= n*m)
    # Y dim(exp(I)) = deg(h) luego deg(HF_{R/I}) = dim(exp(G))
    return dim_ideal(R, F)

In [68]:
# Parcial 2 - Ejercicio 5
R.<x,y,z,t> = PolynomialRing(QQ, 4, "xyzt", order = "deglex")
F = [x^2-y*z+z*t, x^2+y^3-y*z*t, x*y*z-x^2*t+y^2*t^2]
print(grado_Hilbert(R, F))

1


In [65]:
# Minímo s0 donde coincide h(s) = HF_E(s)
def min_s0(R, F):
    n = len(R.gens())
    if 1 in F: return 0
    alphas = exp(R.ideal(F).groebner_basis())
    return n * max([max(alpha) for alpha in alphas])

In [72]:
R.<x,y,z,t> = PolynomialRing(QQ, 4, "xyzt", order = "deglex")
F = [x^2-y*z+z*t, x^2+y^3-y*z*t, x*y*z-x^2*t+y^2*t^2]
print(min_s0(R, F))

12


In [71]:
R.<x,y> = PolynomialRing(QQ, 2, "xy", order = "deglex")
F = [x^3*y, x*y^2]
print(R.ideal(F).groebner_basis())
print(min_s0(R,F))

[x^3*y, x*y^2]
6
